In [1]:
import pandas as pd
from bs4 import BeautifulSoup as bs
import requests


import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect

In [2]:
engine = create_engine('sqlite://', echo=False)

In [3]:
url= "https://www.litzusa.com/zh-HK/StudyusaRecords/detail/US-News-National-University-Ranking-%e7%be%8e%e5%9c%8b%e5%a4%a7%e5%ad%b8%e6%8e%92%e5%90%8d"
response = requests.get(url)

In [4]:
soup = bs(response.text, 'lxml')
print(soup)

<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">
<html xmlns="http://www.w3.org/1999/xhtml">
<head>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/> <title>US News &amp; World Report - National Universities ranking 2020,2020年度美國新聞美國大學排名|美國升學顧問|美國留學途徑|升學展覽/留學講座|美國名校大學資料|Study USA</title>
<meta content="US News &amp; World Report, National Universities Ranking,美國大學排名,美國新聞及世界日報,升學顧問,美國升學,美國留學,海外升學,升學展覽,升學講座,海外留學,留學中心,留學途徑,美國中學,美國學位,美國學費,美國大學,美國學制,美國名校,交流生,美國Visa,教育制度,社區學院,美國州立大學,Study USA,US High School,US College,US University,私立中學,寄宿學校,TOEFL,SAT,IELTS,加州,Foothill學院,華盛頓,Edmonds學院" name="keyword"/>
<meta content="U.S. News &amp; World Report - National University Ranking 2021,美國新聞及世界日報-2021年美國大學排名。|美國升學/留學展覽,海外留學途徑/升學講座,美國名校,美國中學US High School,社區學院Community College,美國州立大學US State University,交流生計劃,私立中學,寄宿學校,美國教育制度/學制/學位/大學資料" name="description"/>
<meta content="width=device-width, initial-scale=1, 

In [5]:
results= soup.find('div', class_='right_container')
print(results)

<div class="right_container">
<h1 class="pageTitle">各類排名<div class="arrow"></div></h1>
<div class="bordershadow"></div>
<a class="backbtn link-back" href="javascript:history.back();">返回</a> <div class="information-records clearfix">
<h2>2021年度美國新聞及世界日報 - 美國大學排名 2021 US News &amp; World Report - National Universities rankings </h2>
<p><span data-mce-mark="1" style="color: #6c6c6c; font-size: medium;"><strong><a href="https://www.litzusa.com/zh-HK/StudyusaRecords/detail/University-Ranking-Index-Table-of-content-page" target="_blank" title="Academic Rankings index "><img alt="/uploaded/rankings-star-icon.jpg" height="67" src="/uploaded/rankings-star-icon.jpg" width="75"/></a> <span style="font-size: small;">&lt;- <strong>返回排名的索引頁</strong></span></strong></span></p>
<p> <strong style="color: #6c6c6c; font-size: medium;">2021年 US News美國大學排名</strong></p>
<p><strong style="color: #6c6c6c; font-size: medium;">Top 70 </strong></p>
<table border="0" cellpadding="0" cellspacing="0" style="width: 

In [6]:
body = soup.find_all("tr")
# print(body)
all_schools = [] # will be a list for list for all schools
for row_num in range(181): # A row at a time (Page is one big table, only need 180 USNWR + Header)
    school = [] # this will old entries for one school
    for row_item in body[row_num].find_all("td"): #loop through all row entries
        # row_item.text removes the tags from the entries
        aa = row_item.text.strip()#.encode('utf-8', 'ignore')
        #append aa to row - note one row entry is being appended
        school.append(aa)
    # append one row to all_rows
    all_schools.append(school)

In [7]:
print(aa)
# print(school)
# print(all_schools)

Montclair, NJ


In [8]:
#Make DF
df = pd.DataFrame(all_schools)
#headers are first row
df.columns = df.iloc[0]
#drop that row now that they're headers
df=df.drop(df.index[0])
df.head()

,Rank#,Unviersity,Location
1,1,Princeton University,"Princeton, NJ"
2,2,Harvard University,"Cambridge, MA"
3,3,Columbia University,"New York, NY"
4,4,Massachusetts Institute of Technology,"Cambridge, MA"
5,4,Yale University,"New Haven, CT"


In [9]:
# Break out State from City
# new data frame with split value columns
new = df["Location"].str.split(",", n = 1, expand = True)
 
# making separate state name column from new data frame
# df["City"]= new[0]
df["StateAbv"]= new[1]
  
# Dropping old Name columns and Rank which we don't need
df.drop(columns =["Location", "Rank#"], inplace = True)
#reset index just to be sure for merge

df.head()

,Unviersity,StateAbv
1,Princeton University,NJ
2,Harvard University,MA
3,Columbia University,NY
4,Massachusetts Institute of Technology,MA
5,Yale University,CT


In [10]:
dfbystate=df.groupby("StateAbv").count()
dfbystate.head()

,Unviersity
StateAbv,
AL,4
AR,1
AZ,2
CA,21
CO,4


In [11]:
dfbystate= dfbystate.reset_index()
dfbystate.head()

,StateAbv,Unviersity
0,AL,4
1,AR,1
2,AZ,2
3,CA,21
4,CO,4


In [12]:
dfbystate.StateAbv=dfbystate.StateAbv.str.strip()

In [13]:
states= pd.read_csv('states.csv')
states=states.rename(columns={"State.1": "StateAbv"}).reset_index(drop=True)
#     .drop(columns=['State Capital','Region'])\
        
states.StateAbv = states.StateAbv.str.strip()
states.convert_dtypes()
states

,State,StateAbv,State Capital,Region
0,Alabama,AL,Montgomery,South
1,Alaska,AK,Juneau,West
2,Arizona,AZ,Phoenix,West
3,Arkansas,AR,Little Rock,South
4,California,CA,Sacramento,West
5,Colorado,CO,Denver,West
6,Connecticut,CT,Hartford,Northeast
7,Delaware,DE,Dover,Northeast
8,Florida,FL,Tallahassee,South
9,Georgia,GA,Atlanta,South


In [14]:
alldfbystate.merge(states, how='left', on='StateAbv')


,StateAbv,Unviersity,State,State Capital,Region
0,AL,4,Alabama,Montgomery,South
1,AR,1,Arkansas,Little Rock,South
2,AZ,2,Arizona,Phoenix,West
3,CA,21,California,Sacramento,West
4,CO,4,Colorado,Denver,West
5,CT,3,Connecticut,Hartford,Northeast
6,DC,6,District of Columbia,Washington,Northeast
7,DE,1,Delaware,Dover,Northeast
8,FL,5,Florida,Tallahassee,South
9,GA,4,Georgia,Atlanta,South


In [15]:
dfbystate.to_csv("colleges.csv", index=False)